In [16]:
import pandas as pd
import numpy as np

In [17]:
df=pd.read_csv("train.csv")

In [18]:
df.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [19]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales'],
      dtype='object')

In [20]:
df=df[["Order Date","Product ID","Sales"]]

In [21]:
df.isnull().sum()

Order Date    0
Product ID    0
Sales         0
dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Order Date  9800 non-null   object 
 1   Product ID  9800 non-null   object 
 2   Sales       9800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 229.8+ KB


In [23]:

DATE_COL = "Order Date"
ID_COL="Product ID"
TARGET_COL = "Sales"
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
FORECAST_HORIZON=18



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '15/04/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '22/11/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '13/05/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Library/Frameworks/Python.framework/Versions/3.9/lib/p

In [24]:
from datetime import date, timedelta
monthly_aggrigated_ecommerce_data=pd.DataFrame()
        
        
for article in df[ID_COL].unique():
    sub_df = df[(df[ID_COL]==article)]
    #end_date_last_day = sub_df[DATE_COL].max().replace(day=1) - timedelta(days=1)
    

    end_date_last_day = date.today().replace(day=1) - timedelta(days=1)
    
    end_date_last_day=pd.to_datetime(end_date_last_day)


#filter out the transactions or drop current month demand as it is progressively get updated
    sub_df = sub_df[(sub_df[DATE_COL] <= end_date_last_day)]
    if not(sub_df.empty):
        sub_df_copy = sub_df.copy()
        #add a record to fill 0s for discontinued product untill last month
        if(sub_df[DATE_COL].max() < end_date_last_day):
            record_to_insert = {ID_COL: article, DATE_COL: end_date_last_day, TARGET_COL : 0} #populate a dummy record of last month so that resampling function handles filling 0s for discontinued products 
            sub_df = sub_df.append(record_to_insert, ignore_index=True)

        #sub_df = sub_df.append(generate_forecast_window(end_date_last_day, FORECAST_HORIZON, article, customer, cost=sub_df["COST"].iloc[-1]), ignore_index=True)
        #df['date'] = pd.to_datetime(df['date'])
        sub_df[DATE_COL]=pd.to_datetime(sub_df[DATE_COL])
        sub_df.index = sub_df[DATE_COL]
        sub_df = sub_df.resample(rule='MS').agg({'Sales' : 'sum'})
        sub_df = sub_df.reset_index()
        sub_df[ID_COL] = article  
        monthly_aggrigated_ecommerce_data = monthly_aggrigated_ecommerce_data.append(sub_df, ignore_index=True)

monthly_aggrigated_ecommerce_data.to_csv("output.csv",index = False)


/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/639018602.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_df = sub_df.append(record_to_insert, ignore_index=True)
/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/639018602.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  monthly_aggrigated_ecommerce_data = monthly_aggrigated_ecommerce_data.append(sub_df, ignore_index=True)
/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/639018602.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sub_df = sub_df.append(record_to_insert, ignore_index=True)
/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/639018602.py:31: FutureWarning: The frame.append method is d

In [25]:

def split_time_series_for_modeling(df, evaluation_size):
            import pandas as pd 
            import numpy as np
            train_size = len(df) - evaluation_size
            train, eval = df.iloc[:train_size], df.iloc[train_size:]
            return train, eval
train_df1=pd.DataFrame()
eval_df1=pd.DataFrame()
        
for article in monthly_aggrigated_ecommerce_data[ID_COL].unique():
    single_article_df = monthly_aggrigated_ecommerce_data[monthly_aggrigated_ecommerce_data[ID_COL] == article]
    if not(single_article_df.empty):
        train_df, evaluation_df = split_time_series_for_modeling(single_article_df, evaluation_size = 6) #Need to applied in component 2 and only train, test dfs need to be there
        train_df1=train_df1.append(train_df)
        eval_df1=eval_df1.append(evaluation_df)
    #train_df1.to_csv(train_df_path.path,index = False)
    #eval_df1.to_csv(validation_df_path.path,index = False)


/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/3231026087.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df1=train_df1.append(train_df)
/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/3231026087.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval_df1=eval_df1.append(evaluation_df)
/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/3231026087.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df1=train_df1.append(train_df)
/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/3231026087.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval_df1=eval_

In [26]:
train_df1


,Order Date,Sales,Product ID
0,2017-08-01,261.960,FUR-BO-10001798
1,2017-09-01,556.665,FUR-BO-10001798
2,2017-10-01,0.000,FUR-BO-10001798
3,2017-11-01,0.000,FUR-BO-10001798
4,2017-12-01,0.000,FUR-BO-10001798
...,...,...,...
144291,2021-07-01,0.000,TEC-PH-10002645
144292,2021-08-01,0.000,TEC-PH-10002645
144293,2021-09-01,0.000,TEC-PH-10002645
144294,2021-10-01,0.000,TEC-PH-10002645


In [27]:
for article in monthly_aggrigated_ecommerce_data[ID_COL].unique():
    single_article_training_df =train_df1[train_df1[ID_COL] == article]
    single_article_evaluation_df = eval_df1[eval_df1[ID_COL] == article]
   
    article_df=single_article_training_df.append(single_article_evaluation_df,ignore_index=True)


/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/2243902538.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  article_df=single_article_training_df.append(single_article_evaluation_df,ignore_index=True)


In [28]:
monthly_aggrigated_ecommerce_data =monthly_aggrigated_ecommerce_data[monthly_aggrigated_ecommerce_data[ID_COL] == "TEC-PH-10002645"]

In [29]:
article_df

,Order Date,Sales,Product ID
0,2017-12-01,1499.97,TEC-PH-10002645
1,2018-01-01,0.00,TEC-PH-10002645
2,2018-02-01,0.00,TEC-PH-10002645
3,2018-03-01,0.00,TEC-PH-10002645
4,2018-04-01,0.00,TEC-PH-10002645
5,2018-05-01,0.00,TEC-PH-10002645
6,2018-06-01,0.00,TEC-PH-10002645
7,2018-07-01,0.00,TEC-PH-10002645
8,2018-08-01,0.00,TEC-PH-10002645
9,2018-09-01,0.00,TEC-PH-10002645


In [61]:

from statsmodels.tsa.ar_model import AutoReg

def ceil_list_of_values(values):
    return np.ceil(values)

def generate_dates(last_date, horizon):
    print("generate dates started!!!")
    dates_list = list(pd.date_range(start=last_date , periods=horizon + 1,freq="M"))
    print("dates SUCESSFUL!!!")
    return dates_list[1:]

def generate_algorithm_results_df(eval_df, model_predictions, model_forecasts, algorithm_name):
        FORECAST_HORIZON=18
        print("algorithem started!!!")
        eval_df[TARGET_COL+"_PREDICTED"] = model_predictions
        last_date = eval_df[DATE_COL].values.tolist()[-1] #last date
        projected_dates = generate_dates(last_date, FORECAST_HORIZON)
        forecast_df = pd.DataFrame()
        forecast_df[DATE_COL] = projected_dates
        forecast_df[ID_COL] = eval_df[ID_COL].values[0]
        #forecast_df["CUSTOMER_NAME"] = eval_df["CUSTOMER_NAME_"].values[0] #comment for primary and secondary retail
        forecast_df[TARGET_COL+"_Forecasted"] = np.ceil(model_forecasts)
        final_outcome_df = pd.concat([eval_df, forecast_df], axis=0)
        final_outcome_df["MODEL"] = algorithm_name
        print("algorithem SUCESSFUL!!!")
        return final_outcome_df


def ARMODEL(train, test, article_df):
    TARGET_COL="Sales"
    print("model started")
    best_rmse = float('inf')
    best_lag = None
    evaluation_predictions = None
    forecast_predictions = None
    data_length = len(article_df[TARGET_COL].values.tolist())
    lags_list = range(0, data_length // 2)
    print("l")
    for lag in lags_list:
        
        model = AutoReg(train[TARGET_COL].values, lags=lag)
        model_fit = model.fit()
        predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)

        error_rate = np.sqrt(mean_squared_error(test[TARGET_COL].values, predictions))

        #best parameters
        if best_rmse >= error_rate:
            best_rmse = error_rate
            evaluation_predictions = predictions
            print("pred")
            print(predictions)
            best_lag = lag
        else:
            pass

    model = AutoReg(article_df[TARGET_COL].values, lags=best_lag)
    model_fit = model.fit()
    print(model_fit.summary())
    #print("model")
    #print(len(article_df) + 1)
    #print(len(article_df))
    forecast_predictions = model_fit.predict(start=len(article_df) + 1, end=len(article_df) + 18, dynamic=False)  
    print("g") 
    #evaluation_predictions = ceil_list_of_values(evaluation_predictions)
    #forecast_predictions = ceil_list_of_values(forecast_predictions)
    print("done")
    return evaluation_predictions, forecast_predictions
algorithem_df=pd.DataFrame()
for article in monthly_aggrigated_ecommerce_data[ID_COL].unique():
    single_article_training_df =train_df1[train_df1[ID_COL] == article]
    single_article_evaluation_df = eval_df1[eval_df1[ID_COL] == article]
    try:
        
        article_df=single_article_training_df.append(single_article_evaluation_df,ignore_index=True)

        model_evaluation_predictions, model_forecast_predictions = ARMODEL(single_article_training_df, single_article_evaluation_df, article_df)
        print("fg")
        algorithm_results = generate_algorithm_results_df(single_article_evaluation_df, model_evaluation_predictions, model_forecast_predictions, "AR")
        algorithem_df=algorithem_df.append(algorithm_results,ignore_index=True)
        algorithem_df.to_csv("algo_df.csv",index = False)

        print(algorithem_df.head())
        
        
    except Exception as e:
        print(e)
        continue


model started
l
name 'mean_squared_error' is not defined


/var/folders/sl/mghz4g4x4nq06lrj3hn18prw0000gn/T/ipykernel_14852/2562949291.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  article_df=single_article_training_df.append(single_article_evaluation_df,ignore_index=True)


In [59]:
algorithem_df

,Order Date,Sales,Product ID,Sales_PREDICTED,Sales_Forecasted,MODEL
0,2021-12-01,0.0,TEC-PH-10002645,None,NaN,AR
1,2022-01-01,0.0,TEC-PH-10002645,None,NaN,AR
2,2022-02-01,0.0,TEC-PH-10002645,None,NaN,AR
3,2022-03-01,0.0,TEC-PH-10002645,None,NaN,AR
4,2022-04-01,0.0,TEC-PH-10002645,None,NaN,AR
5,2022-05-01,0.0,TEC-PH-10002645,None,NaN,AR
6,2022-06-30,NaN,TEC-PH-10002645,NaN,28.0,AR
7,2022-07-31,NaN,TEC-PH-10002645,NaN,28.0,AR
8,2022-08-31,NaN,TEC-PH-10002645,NaN,28.0,AR
9,2022-09-30,NaN,TEC-PH-10002645,NaN,28.0,AR
